# Example of generating data for training/evaluation
---


## Table of Contents

1. [Data generator for training CFO Correction Network](#cfo_data_generator)
2. [Data generator for training Equalization](#equalization)
3. [Data generator for training Demod + Decoder](#decoder)
4. [Data generator for training End to End](#end2end)

## Environment setup

In [1]:
import os
import sys

import numpy as np

# Import radioml package from other directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)    

# For visualization
import imageio
from IPython.display import Image, display
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
sns.set_style("darkgrid")
%matplotlib inline

## Define Parameters

In [2]:
NUM_PACKETS  = 5
PREAMBLE_LEN = 40
DATA_LEN     = 200
CHANNEL_LEN = 1
OMEGA      = 1/50
SNR_train  = 10.0 

## Visualization Funcs

In [3]:
plt.ioff()


def visualize_signals(ax, x, y, groundtruths=None, title=None, cmap='Set1', min_val=-2, max_val=2):
    ax.scatter(x, y, c=groundtruths, s=100, cmap=cmap)
    ax.set_xlabel('I-component')
    ax.set_ylabel('Q-component')
    ax.set_title(title)
    ax.axhline()
    ax.axvline()
#     ax.set_xlim(min_val, max_val)
#     ax.set_ylim(min_val, max_val)

def visualize_cfo(radio, omega, snr):
    """Visualize the effect of Carrier Offset Frequency."""
    generator = cfo_net_data_generator(radio, omega, snr, batch_size=4)

    [preambles, preambles_conv], cfo_corrected_preamble = next(generator)
    symbols, groundtruths = np.unique(preambles.view(np.complex),return_inverse=True)

    fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(10, 5))
    st = fig.suptitle("Omega = %.3f || SNR_dB = %.2f " % (omega, snr))
    visualize_signals(left_ax, 
                     x=preambles_conv[...,0].flatten(), 
                     y=preambles_conv[...,1].flatten(),
                     groundtruths=groundtruths, 
                     title="Before CFO Correction", 
                     cmap='Set1')
    visualize_signals(right_ax, 
                     x=cfo_corrected_preamble[...,0].flatten(), 
                     y=cfo_corrected_preamble[...,1].flatten(),
                     groundtruths=groundtruths, 
                     title="After CFO Correction")
    img = _convert_fig_to_img(fig)
    return img
    

def _convert_fig_to_img(fig):
    """Convert figure to image."""
    # Used to return the plot as an image rray
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return image

## Example of data generator for training CFO Correction  <a class="anchor" id="cfo_data_generator"></a>


In [ ]:
%%time

from radioml.core import RadioTransmitter
from radioml.dataset import cfo_net_data_generator

channel_len = 1
radio_transmitter = RadioTransmitter(DATA_LEN, PREAMBLE_LEN,'QPSK')
generator = cfo_net_data_generator(radio_transmitter, OMEGA, SNR_train, batch_size=4)

for i in range(3):
    one_batch = next(generator)
    [preambles, convolved_preambles], cfo_corrected_preambles = one_batch

In [ ]:
fnames = []

radios = [RadioTransmitter(0, PREAMBLE_LEN, mod) for mod in  ['QPSK', 'QAM16']]

# Save and Display the animated result 
filename = './figures/cfo_%s.gif' % 'qam16'
imageio.mimsave(filename, 
                [visualize_cfo(radio, omega, 30.0) for radio in radios for omega in [1/100., 1/50, 1/30]], 
                fps=0.5)
    
with open(filename,'rb') as f:
    display(Image(data=f.read(), format='png'))

## Example of data generatorfor training Equalization  <a class="anchor" id="equalization"></a>


In [ ]:
def visualize_equalization(radio, channel_len, snr):
    """Visualize the effect of Symbol Inteference."""
    
    examples = next(equalization_data_generator(radio, 
                                                channel_len,
                                                snr, 
                                                batch_size=1, 
                                                num_cpus=8))
    
    [preambles, _, cfo_corrected_data], \
    [equalized_data, modulated_packet] = examples   
    
    modulated_packet = modulated_packet[:, radio.modulated_preamble_len:]
    symbols, groundtruths = np.unique(modulated_packet.view(np.complex),return_inverse=True)
    fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(10,5))
    st = fig.suptitle("Number of Channels = %d" % channel_len)
    
    visualize_signals(left_ax, 
                      cfo_corrected_data[...,0].flatten(),
                      cfo_corrected_data[...,1].flatten(), 
                      groundtruths,
                      title="Before Equalization", 
                      min_val=-3,max_val=3, cmap='rainbow')
    visualize_signals(right_ax, 
                      equalized_data[...,0].flatten(),
                      equalized_data[...,1].flatten(),
                      groundtruths,
                      title="After Equalization", 
                      min_val=-3, max_val=3)
    img = _convert_fig_to_img(fig)
    return img

In [ ]:
%%time
from radioml.core import RadioTransmitter
from radioml.dataset import equalization_data_generator

radio = RadioTransmitter(DATA_LEN,PREAMBLE_LEN, modulation_scheme='QPSK')
generator = equalization_data_generator(radio, CHANNEL_LEN, SNR_train, batch_size=32)

for i in range(3):
    one_batch = next(generator)
    [preambles, cfo_corrected_preamble, cfo_corrected_preamble], channels_etimates = one_batch

In [ ]:
# Visualize equalization data
fnames=[]
radio = RadioTransmitter(DATA_LEN,PREAMBLE_LEN, modulation_scheme='QPSK')

CHANNEL_TAPS = [2]
for modulation_scheme in ['QPSK']:
    filename = './figures/channel_inference_%s.gif'%modulation_scheme
    imageio.mimsave(filename, 
                    [visualize_equalization(radio, channel_tap, 50.0) 
                      for channel_tap in CHANNEL_TAPS], fps=2)
    fnames.append(filename)

with open(filename,'rb') as f:
    display(Image(data=f.read(), format='png'))

## Data generator for training Demod + Decoder <a class="anchor" id="decoder"></a>

In [4]:
def visualize_demod_n_decode(radio, snr):
    [equalized_data, modulated_data], \
    data_estimate = next(ecc_data_generator(radio, snr, batch_size=4))
    
    symbols, groundtruths = np.unique(modulated_data.view(np.complex), return_inverse=True)
    fig, ax = plt.subplots(1, 1)
    
    visualize_signals(ax, 
                      equalized_data[...,0].flatten(),
                      equalized_data[...,1].flatten(), 
                      groundtruths,
                      title="Equalized singals at $SNR_{dB}$ = %.2f dB"%snr)

    img = _convert_fig_to_img(fig)
    return img

In [5]:
%%time
from radioml.core import RadioTransmitter
from radioml.dataset import ecc_data_generator

radio = RadioTransmitter(DATA_LEN,PREAMBLE_LEN, modulation_scheme='QPSK')
generator = ecc_data_generator(radio, SNR_train, batch_size=32)

for i in range(3):
    one_batch = next(generator)
    equalized_data, data_estimate = one_batch

CPU times: user 2.61 s, sys: 252 ms, total: 2.86 s
Wall time: 3.56 s


In [7]:
SNRs = [13.0, 15.0, 17.0, 20.0, 30.0]

radio = RadioTransmitter(DATA_LEN,PREAMBLE_LEN, modulation_scheme='QPSK')
imageio.mimsave('./figures/error_correction.gif', 
                   [visualize_demod_n_decode(radio, snr) for snr in SNRs], 
                   fps=1)
display(Image(url='./figures/error_correction.gif'))

In [ ]:
%%time
radio = RadioDataGenerator(DATA_LEN,PREAMBLE_LEN, CHANNEL_LEN, modulation_scheme='QPSK')
generator = radio.end2end_data_generator(OMEGA, SNR_train, batch_size=32)

for i in range(3):
    one_batch = next(generator)
    [preambles, corrupted_packets], data_etimates = one_batch